In [24]:
import pandas as pd

from pulp import LpProblem, LpMaximize, lpSum, LpVariable, LpStatus, LpInteger, LpBinary

data = pd.read_excel("data.xls").dropna()

In [25]:
data

,Code,Name,Club,Value,PLD,GLS,ASS,CS,GA,Pts,Position
0,132,Alisson,LIV,4.0,28.0,0.0,1.0,14.0,23.0,35,GK
1,118,Ederson,MC,4.0,34.0,0.0,0.0,16.0,28.0,38,GK
2,140,N. Pope,BUR,3.8,38.0,0.0,0.0,15.0,50.0,18,GK
3,126,K. Schmeichel,LEI,3.7,38.0,0.0,0.0,13.0,41.0,23,GK
4,135,D. De Gea,MU,3.7,38.0,0.0,0.0,13.0,36.0,28,GK
...,...,...,...,...,...,...,...,...,...,...,...
452,601,F. Andone,BHA,5.6,0.0,1.0,0.0,0.0,0.0,3,ST
454,643,Y. Muto,NEW,5.6,2.0,0.0,0.0,0.0,0.0,0,ST
455,666,K. Davis,AV,5.5,4.0,0.0,0.0,0.0,0.0,0,ST
457,615,C. Wilson,NEW,6.7,33.0,8.0,4.0,0.0,0.0,32,ST


In [26]:
# Create the data groups

PLAYERS = data['Name'].tolist()

TEAMS = {
    team: data[data['Club'] == team]['Name'].tolist() for team in data['Club'].unique().tolist()
}

POSITIONS = {
    position: data[data['Position'] == position]['Name'].tolist() for position in data['Position'].unique().tolist()
}

values = {
    player: data[data['Name'] == player].iloc[0]['Value'] for player in PLAYERS
}

points = {
    player: data[data['Name'] == player].iloc[0]['Pts'] for player in PLAYERS
}


In [27]:
# Create problem

model = LpProblem("Fantasy", LpMaximize)

# variables
position_constraints = {
    'GK': 1,
    'FB': 2,
    'CB': 2,
    'MF': 4,
    'ST': 2
}

# Create player variables

player_vars = LpVariable.dicts("PlayerVariables", [(i) for i in PLAYERS], 0, 1, LpBinary)

## CONSTRAINTS

# Value constraint
model += lpSum(player_vars[(i)] * values[i] for i in PLAYERS) <= 50.0

# Team constraint
for team, players in TEAMS.items():
    model += lpSum(player_vars[(i)] for i in players) <= 2
    
# Position constraint

for position, players in POSITIONS.items():
    model += lpSum(player_vars[(i)] for i in players) == position_constraints[position]
    
## OBJECTIVE FUNCTION
model += lpSum(player_vars[(i)] * points[i] for i in PLAYERS)


## SOLVE
model.solve()
    

1

In [28]:
LpStatus[model.status]

'Optimal'

In [29]:
for v in model.variables():
    if v.varValue == 1:
        print(v.name)

PlayerVariables_A._El_Ghazi
PlayerVariables_C._Basham
PlayerVariables_D._De_Gea
PlayerVariables_D._Ings
PlayerVariables_H._Barnes
PlayerVariables_J._Egan
PlayerVariables_J._Vardy
PlayerVariables_K._De_Bruyne
PlayerVariables_M._Doherty
PlayerVariables_R._Snodgrass
PlayerVariables_T._Alexander_Arnold
